In [18]:
##DAG1 with logging
 
import logging

from snowflake.snowpark import Session
 
# Set up logging with StreamHandler to output to console (in case the notebook is capturing logs differently)

logger = logging.getLogger()

logger.setLevel(logging.INFO)  # Set log level to INFO
 
# Create a StreamHandler to output logs to the notebook's console

console_handler = logging.StreamHandler()

console_handler.setLevel(logging.INFO)
 
# Create a formatter for better log format

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

console_handler.setFormatter(formatter)
 
# Add the handler to the logger

logger.addHandler(console_handler)
 
def my_stored_procedure(session: Session):

    try:

        logger.info("Setting role to ACCOUNTADMIN...")

        session.sql("USE ROLE ACCOUNTADMIN").collect()

        logger.info("Role set to ACCOUNTADMIN successfully.")
 
        # Create the table

        logger.info("Creating table RAW_CO2.Daily_Measurements...")

        create_table_query = """

        CREATE OR REPLACE TABLE RAW_CO2.Daily_Measurements (

            date STRING,

            co2_ppm FLOAT

        );

        """

        session.sql(create_table_query).collect()

        logger.info("Table RAW_CO2.Daily_Measurements created successfully.")
 
        # Grant privileges on the table to ACCOUNTADMIN role

        logger.info("Granting privileges on table to ACCOUNTADMIN...")

        grant_privileges_query = """

        GRANT ALL PRIVILEGES ON TABLE RAW_CO2.Daily_Measurements TO ROLE ACCOUNTADMIN;

        """

        session.sql(grant_privileges_query).collect()

        logger.info("Privileges granted to ACCOUNTADMIN successfully.")
 
        # Copy data from the stage into the table

        logger.info("Copying data from the stage into RAW_CO2.Daily_Measurements...")

        copy_query = """

        COPY INTO RAW_CO2.Daily_Measurements

            FROM @RAW_CO2.CO2_EXTERNAL_STAGE

            FILE_FORMAT = (

                TYPE = 'CSV' 

                SKIP_HEADER = 1

                FIELD_OPTIONALLY_ENCLOSED_BY = '"'

            )

            ON_ERROR = 'CONTINUE';

        """

        session.sql(copy_query).collect()

        logger.info("Data copied into RAW_CO2.Daily_Measurements successfully.")
 
        # Create a stream on the table

        logger.info("Creating stream on RAW_CO2.Daily_Measurements...")

        create_stream_query = """

        CREATE OR REPLACE STREAM RAW_CO2.DAILY_MEASUREMENTS_STREAM 

        ON TABLE RAW_CO2.Daily_Measurements;

        """

        session.sql(create_stream_query).collect()

        logger.info("Stream RAW_CO2.DAILY_MEASUREMENTS_STREAM created successfully.")
 
        # Optionally, verify if the table, privileges, and stream were created successfully

        logger.info("Table, privileges, and stream are created successfully.")
 
    except Exception as e:

        logger.error(f"An error occurred: {str(e)}")

        raise  # Reraise the exception to propagate it
 
# Call the stored procedure with the appropriate session (handled automatically by Snowflake in the notebook)

my_stored_procedure(session)
 

SyntaxError: invalid syntax (1166821928.py, line 1)